<a href="https://colab.research.google.com/github/1BM23CS308/1BM23CS308_BIS_LAB/blob/main/software_test_caseoptimization(CSA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import random
import math

test_cases = [
    [5, 10],
    [3, 6],
    [6, 14],
    [2, 4],
    [7, 12]
]

num_cases = len(test_cases)
num_nests = 10
Pa = 0.25
MaxIter = 50

def fitness(sequence):
    total_faults = 0
    total_time = 0
    for idx in sequence:
        total_faults += test_cases[idx][0]
        total_time += test_cases[idx][1]
    return total_faults / total_time

def levy_flight(Lambda):
    sigma = (math.gamma(1 + Lambda) * math.sin(math.pi * Lambda / 2) /
            (math.gamma((1 + Lambda) / 2) * Lambda * 2 ** ((Lambda - 1) / 2))) ** (1 / Lambda)
    u = np.random.normal(0, sigma)
    v = np.random.normal(0, 1)
    step = u / abs(v) ** (1 / Lambda)
    return step

def generate_new_sequence(current_seq):
    seq = current_seq.copy()
    step = abs(int(levy_flight(1.5))) % num_cases
    if step > 0:
        i, j = random.sample(range(num_cases), 2)
        seq[i], seq[j] = seq[j], seq[i]
    return seq

nests = [random.sample(range(num_cases), num_cases) for _ in range(num_nests)]
fitness_values = [fitness(seq) for seq in nests]

best_nest = nests[np.argmax(fitness_values)]
best_fitness = max(fitness_values)

for t in range(MaxIter):
    for i in range(num_nests):
        new_seq = generate_new_sequence(nests[i])
        new_fit = fitness(new_seq)
        if new_fit > fitness_values[i]:
            nests[i] = new_seq
            fitness_values[i] = new_fit
    abandon_count = int(Pa * num_nests)
    worst_indices = np.argsort(fitness_values)[:abandon_count]
    for idx in worst_indices:
        nests[idx] = random.sample(range(num_cases), num_cases)
        fitness_values[idx] = fitness(nests[idx])
    current_best_idx = np.argmax(fitness_values)
    if fitness_values[current_best_idx] > best_fitness:
        best_nest = nests[current_best_idx]
        best_fitness = fitness_values[current_best_idx]

print("Best Test Sequence:", [f"TC{idx+1}" for idx in best_nest])
print("Maximum Fitness:", round(best_fitness, 4))

Best Test Sequence: ['TC3', 'TC2', 'TC5', 'TC4', 'TC1']
Maximum Fitness: 0.5
